# XGBoost Training with Connolly Point arrays
We begin by setting up XGBoost training with GPU acceleration to speed up the process and handle large datasets efficiently.

In [ ]:
# == Libraries ==
import os
import numpy as np
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from optuna.integration import XGBoostPruningCallback
from datetime import timedelta
import time



## Step 1: Load and Merge Feature Batches
We load 10 NumPy arrays (batches) containing Connolly points represented as 32 features and 1 label. Although batching was useful during preprocessing, merging them now is better for training the ML model as a whole.

In [ ]:

# ===  Load and split all batches (Train/Test only) ===
INPUT_FOLDER = "input_numpy"
N_BATCHES = 10

X_train_all, y_train_all = [], []
X_test_all, y_test_all = [], []

for i in range(1, N_BATCHES + 1):
    batch_path = os.path.join(INPUT_FOLDER, f"batch_{i}.npy")
    data = np.load(batch_path)
    X = data[:, :-1]
    y = data[:, -1]

    # Stratified split: 70% train, 30% test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )

    X_train_all.append(X_train)
    y_train_all.append(y_train)
    X_test_all.append(X_test)
    y_test_all.append(y_test)

# Merge all batches
X_train_all = np.concatenate(X_train_all)
y_train_all = np.concatenate(y_train_all)
X_test_all = np.concatenate(X_test_all)
y_test_all = np.concatenate(y_test_all)




## Step 2: Explore Dataset and Class Distribution
Let's inspect the dataset to understand its distribution. Spoiler: there's a significant class imbalance between binding site and non-binding site Connolly points.

In [12]:
# === Optional: Print dataset summary ===
total = len(y_train_all) + len(y_test_all)
print("Dataset Summary")
print(f"Total samples: {total:,}")
print(f"Train set: {len(y_train_all):,} samples | Positives: {(y_train_all == 1).sum():,} ({(y_train_all == 1).mean()*100:.2f}%)")
print(f"Test set:  {len(y_test_all):,} samples | Positives: {(y_test_all == 1).sum():,} ({(y_test_all == 1).mean()*100:.2f}%)")


Dataset Summary
Total samples: 11,205,983
Train set: 7,844,183 samples | Positives: 426,817 (5.44%)
Test set:  3,361,800 samples | Positives: 182,921 (5.44%)


## Step 3: Define Hyperparameter Tuning Strategy
We configure Optuna to explore a wide hyperparameter space. scale_pos_weight is especially crucial due to the class imbalance. Optuna uses a Bayesian optimization approach well-suited for XGBoost. We also use the whole train dataset with a 4 fold cross validation.

In [ ]:
import gc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import numpy as np

# Use full training dataset with 4 folds
X_sampled = X_train_all
y_sampled = y_train_all

def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'eval_metric': 'auc',
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
        'gamma': trial.suggest_float('gamma', 0, 2),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 4),
        'reg_lambda': trial.suggest_float('lambda', 0, 2),
        'reg_alpha': trial.suggest_float('alpha', 0, 2),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 5.0, 25.0),
    }

    kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    aucs = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_sampled, y_sampled)):
        X_train_fold, X_val_fold = X_sampled[train_idx], X_sampled[val_idx]
        y_train_fold, y_val_fold = y_sampled[train_idx], y_sampled[val_idx]

        model = xgb.XGBClassifier(**params, n_jobs=2, verbosity=0)

        try:
            model.fit(
                X_train_fold, y_train_fold,
                eval_set=[(X_val_fold, y_val_fold)],
                verbose=False
            )
            preds = model.predict_proba(X_val_fold)[:, 1]
            auc = roc_auc_score(y_val_fold, preds)
            aucs.append(auc)
        except Exception as e:
            print(f"Trial failed on fold {fold}: {e}")
            return 0.0
        finally:
            del model
            gc.collect()

    trial.set_user_attr("fold_aucs", aucs)
    return np.mean(aucs)




## Step 4: Run Extensive Hyperparameter Tuning
We launch Optuna's optimization with 100 trials. This comprehensive tuning process took approximately 4.5 hours with GPU support.

In [ ]:

study = optuna.create_study(direction="maximize")
start_time = time.time()

N_TRIALS = 100

for i in range(N_TRIALS):
    trial_start = time.time()
    study.optimize(objective, n_trials=1, show_progress_bar=False)
    trial_time = time.time() - trial_start

    total_elapsed = time.time() - start_time
    avg_per_trial = total_elapsed / (i + 1)
    remaining = N_TRIALS - (i + 1)
    eta = timedelta(seconds=int(avg_per_trial * remaining))

    print(f"Trial {i+1}/{N_TRIALS} completed in {trial_time:.1f}s | ETA: {eta}")



## Step 5: Best Trial Output
Trial 94 yielded the best results with a score of 0.8116. Below are its optimized parameters, which will be used for final model training and evaluation.


Trial 94 finished with value: 0.8116306670007959 and parameters: {'max_depth': 6, 'learning_rate': 0.2948641667274368, 'n_estimators': 300, 'subsample': 0.871795472197383, 'colsample_bytree': 0.818803808976565, 'gamma': 1.883267048382789, 'min_child_weight': 1, 'lambda': 1.5493470554947497, 'alpha': 0.45298380003513483, 'scale_pos_weight': 13.136280206298453}. Best is trial 94 with value: 0.8116306670007959. 

We will train the model in the entire training set with the best parameters


In [ ]:
# === Train final model on entire training set with best params ===
best_params = study.best_params.copy()
best_params.update({
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda'  
})

final_model = xgb.XGBClassifier(**best_params)

# Train on all training data (no validation split, no early stopping)
final_model.fit(X_train_all, y_train_all, verbose=True)

# === Evaluate on test set ===
from sklearn.metrics import roc_auc_score

test_preds = final_model.predict_proba(X_test_all)[:, 1]
test_auc = roc_auc_score(y_test_all, test_preds)
print(f"Final Test AUC: {test_auc:.4f}")


Final Test AUC: 0.8133


## Step 6: Evaluate Model Performance
While not perfect, our tuned model shows solid performance on this challenging task. Let’s dive into detailed evaluation metrics to assess model quality.

In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Convert probabilities to binary predictions
test_labels = (test_preds >= 0.5).astype(int)

# Compute standard metrics
test_accuracy = accuracy_score(y_test_all, test_labels)
test_precision = precision_score(y_test_all, test_labels, zero_division=0)
test_recall = recall_score(y_test_all, test_labels, zero_division=0)
test_f1 = f1_score(y_test_all, test_labels, zero_division=0)
test_conf_matrix = confusion_matrix(y_test_all, test_labels)
test_report = classification_report(y_test_all, test_labels, digits=4)

print("\nFinal Test Metrics:")
print(f"AUC:       {test_auc:.4f}")
print(f"Accuracy:  {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1 Score:  {test_f1:.4f}")
print("\nConfusion Matrix:\n", test_conf_matrix)
print("\nFull Classification Report:\n", test_report)



✅ Final Test Metrics:
AUC:       0.8133
Accuracy:  0.8309
Precision: 0.1819
Recall:    0.6025
F1 Score:  0.2795

Confusion Matrix:
 [[2683276  495603]
 [  72714  110207]]

Full Classification Report:
               precision    recall  f1-score   support

         0.0     0.9736    0.8441    0.9042   3178879
         1.0     0.1819    0.6025    0.2795    182921

    accuracy                         0.8309   3361800
   macro avg     0.5778    0.7233    0.5918   3361800
weighted avg     0.9305    0.8309    0.8702   3361800



Very low precision -> a lot of false positives, but we prefer getting the most points that really do belong in a binding site even if we get a lot of false positives, because the outlier Connolly points will be filtered out later.

## Step 8: Analyze Feature Importances
We now extract feature importances from the trained model to understand which features contribute most to binding site prediction.

In [ ]:
import pandas as pd
import numpy as np

# === Show feature importances ===
importances = final_model.feature_importances_
importance_df = pd.DataFrame({
    "Feature Index": np.arange(len(importances)),
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

display(importance_df)



,Feature Index,Importance
15,15,0.121020
1,1,0.066357
20,20,0.054123
24,24,0.052567
31,31,0.050008
30,30,0.046646
29,29,0.043578
25,25,0.039008
11,11,0.031604
3,3,0.029356


Interestingly, the most important feature involves Connolly points that act as both Hydrogen Donors and Acceptors, a promising biochemical signal.

## Step 10: Save Trained Model
We save the trained model in multiple formats (json, pkl, etc.) to ensure compatibility with various downstream tools and reproducibility.

In [ ]:
from joblib import dump
import time

# Create a unique timestamped name
timestamp = time.strftime("%Y%m%d_%H%M%S")
model_name = f"xgboost_binding_site_model_{timestamp}"

# Save as native XGBoost binary
final_model.save_model(f"{model_name}.json")  # Good for reloading with XGBoost directly

# Save as Python pickle (Joblib format) — great for reuse in any Python project
dump(final_model, f"{model_name}.pkl")

# Save feature importances too (optional)
importance_df.to_csv(f"{model_name}_feature_importances.csv", index=False)

print(f"Model saved as {model_name}.json and {model_name}.pkl")
